# Implied Volatility and Greeks of 'At The Money' Index Options, Live & Expired: Index_Historical_Option_Greeks

For all the details on the code below, please refer to the article "[Equity Derivatives Intraday Analytics: using Python & IPA to gather intraday insights on live and expired derivatives
](https://developers.lseg.com/en/article-catalog/article/equity-derivatives-intraday-analytics-p-1-using-python-to-gather-intraday-insights-on-live-and-expired-derivatives)".


This notebook demonstrates how to use the 'IPA_Equity_Vola_n_Greeks_Class.py' helper file. It, itself, uses the Refinitiv Data Library for Python to collect data and compute Greeks, Implied Volatilities and Smiles for Equity Options of all kinds; you can use this Notebook to access these functions and display results in a trader-friendly way.
The Python Class from this helper file (i) automatically find the Option (of choice), live or expired, closest to [At The Money (ATM)](https://www.investopedia.com/terms/a/atthemoney.asp) and (ii) calculate its [Implied Volatility](https://www.investopedia.com/terms/i/iv.asp) & [Greeks](https://www.investopedia.com/terms/g/greeks.asp).
We focus below on Future (Monthly) Options on the Index '.STOXX50E' (EURO STOXX 50 EUR PRICE INDEX) ('[EUREX](https://go.refinitiv.com/?u=Y3B1cmw6Ly9hcHBzLmNwLi9BcHBzL0luZGV4Lz9zPS5TVE9YWDUwJnN0PVJJQw%3D%3D&title=.STOXX50%20OV&key=bUq%2BNCiQGAcHQjSNEkJSlZLTgjo9cios8rz4heIBiU8%3D)') and '.SPX' ([S&P 500 INDEX](https://go.refinitiv.com/?u=Y3B1cmw6Ly9hcHBzLmNwLi9BcHBzL0luZGV4Lz9zPS5TUFgmc3Q9UklDIy8%3D&title=.SPX%20OV&key=21C0FJ4FZsZ4s9b6HR4mDSAIHId9rxFOgaCjslcqIVk%3D)), although you can apply the logic below for another index.
To find the ATM instrument, we simply and efficiently use the [Search API](https://developers.refinitiv.com/en/article-catalog/article/building-search-into-your-application-workflow).
Usually, the calculation of the Black-Scholes-Merton model's Implied Volatility involves numerical techniques, since it is not a [closed equation](https://en.wikipedia.org/wiki/Closed-form_expression) ([unless restricting assumptions that log returns follow a standard normal distribution with mean is zero, $\mu$ = 0, and standard deviation is zero, $\sigma$ = 1, are made](https://corporatefinanceinstitute.com/resources/derivatives/black-scholes-merton-model/)). If we used these techniques in calculating each Implied Volatility value on our computer, it would take several seconds - if not minutes - for each data point computed. I have chosen to use the [Instrument Pricing Analytics (IPA)](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-option-contracts-eti) service in the Python [Refinitiv Data Library](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-libraries) instead, as this service allows me to send model specifications (and variables) and receive several (up to 100) computed Implied Volatility values in one go - in a few seconds. Not only does this save a great deal of time, but also many lines of code! Fianlly, we will put it all in one function.
There are 3 layers in the Refinitiv Data Library: [Delivery](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/3-Delivery), [Content](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/2-Content) and [Access](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/1-Access). This helper .py file demonstrates how to use the Definition object of the Delivery layer in conjunction with with the functions `get_data` and `get_history` and the Instrument Pricing Analytics (IPA) product.
The Delivery layer provides a simplified data request interface for REST APIs, designed for FinCoders.

#### IPA

IPA (otherwise known as the Quantitative Analytics (QA) API Family) is a powerful product provided by LSEG through its Data Platform REST APIs and [Data Libraries](https://developers.lseg.com/en/api-catalog?x1=w_products&q1=devportal%3Aproducts%2Frefinitiv-data-libraries) that provides calculators for financial operations.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

 You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help. 

----


In [1]:
from helper_option_volatility_greeks import IPA_Equity_Vola_n_Greeeks

In [2]:
%matplotlib inline
from datetime import datetime, timedelta
import plotly.graph_objects as go # `plotly` is a library used to render interactive graphs
import IPython
import ipywidgets as widgets
from refinitiv_widgets import Select, MultiSelect, Button, TextFieldAutosuggest, Calendar, Select, Loader, TextField
import copy
import pandas as pd
import numpy as np

from dataclasses import dataclass

@dataclass(frozen=True)
class ExceptionData:
    data: str

class MyException(Exception):
    def __init__(self, exception_details: ExceptionData):
        self.details = exception_details
    
    def __str__(self):
        return self.details.data


def Eqty_ATM_Optn_Vola_n_Greeks(debug=False):

    eqty_out = widgets.Output()
    eqty_choice = TextFieldAutosuggest(placeholder='Equity Underlying', filters=['EQ', 'INDX'])
    display(eqty_choice, eqty_out)  # This displays the box

    c_p_select_out = widgets.Output()
    c_p_choice = Select(
        placeholder='Call or Put?', width=300.0)
    c_p_choice.data = [
        {'value': i, 'label': i, 'items': []}
        for i in ['Call', 'Put']]
    # display(c_p_choice, c_p_select_out) # If you want to display each choice box one above the other, you can use this line. I chose `HBox`s instead.

    b_s_select_out = widgets.Output()
    b_s_choice = Select(
        placeholder='Buy or Sell?', width=300.0)
    b_s_choice.data = [
        {'value': i, 'label': i, 'items': []}
        for i in ['Buy', 'Sell']]
    # display(b_s_choice, b_s_select_out) # If you want to display each choice box one above the other, you can use this line. I chose `HBox`s instead.

    # Display both choice boxes (`c_p_choice` & `b_s_choice`) horisontally, next to eachother.
    display(widgets.HBox([c_p_choice, b_s_choice])) # This will display both choice boxes horisontally, next to eachother.

    report_ccy_select_out = widgets.Output()
    report_ccy_choice = Select(
        placeholder='Report Currency?', width=300.0)
    report_ccy_choice.data = [
        {'value': i, 'label': i, 'items': []}
        for i in ['EUR', 'USD', 'GBP', 'JPY']]
    # display(report_ccy_choice, report_ccy_select_out) # If you want to display each choice box one above the other, you can use this line. I chose `HBox`s instead.
    
    option_price_side_select_out = widgets.Output()
    option_price_side_choice = Select(
        placeholder='Option Price Side?', width=300.0)
    option_price_side_choice.data = [
        {'value': i, 'label': i, 'items': []}
        for i in ['Let Program Choose', 'Bid', 'Ask']]
    # display(option_price_side_choice, option_price_side_select_out) # If you want to display each choice box one above the other, you can use this line. I chose `HBox`s instead.

    # Display both choice boxes (`report_ccy_choice` & `option_price_side_choice`) horisontally, next to eachother.
    display(widgets.HBox([report_ccy_choice, option_price_side_choice]))

    print("\n")
    print("Please enter the RIC of the reference Risk Free Rate, e.g.: for `.SPX`, go with `USDCFCFCTSA3M=`; for `.STOXX50E`, go with `EURIBOR3MD=`")
    rsk_free_rate_prct_out = widgets.Output()
    rsk_free_rate_prct_choice = TextFieldAutosuggest(placeholder='Risk Free Instrument RIC', filters=['FX'])
    display(rsk_free_rate_prct_choice, rsk_free_rate_prct_out)  # This displays the box


    print("Maturity (note that most Options mature on the third Friday of the month):")
    calendar = Calendar(
        max=(datetime.now() + timedelta(days=30*5)).strftime('%Y-%m-%d'),
        min=(datetime.now() - timedelta(days=30*5)).strftime('%Y-%m-%d'))
    display(calendar)


    widgets.DatePicker(
        description='Start date:', continuous_update=False, max=(datetime.now() + timedelta(days=30*5)).strftime('%Y-%m-%d'))

    # create widgets
    button = Button('Create/Update Graph')
    button_output = widgets.Output()

    loader = Loader(visible=False)
    loader.visible = not loader.visible

    # create click handler
    def click_handler(a):
        with button_output:
            IPython.display.clear_output(wait=True)

            display(loader)
            
            if c_p_choice.value == "" or eqty_choice.value == "" or rsk_free_rate_prct_choice.value == "" or calendar.value == []:
                IPython.display.clear_output(wait=True)
                raise ValueError("Please make sure to complete all fields before running the program.")

            else:

                if debug:
                    print(f"eqty_choice.value: {eqty_choice.value}")
                    print(f"calendar.value[0]: {calendar.value[0]}")
                    print(f"c_p_choice.value: {c_p_choice.value}")
                    print(f"rsk_free_rate_prct_choice.value: {rsk_free_rate_prct_choice.value}")

                print("This may take a few minutes...")

                # Above, we created an option for the `option_price_side_choice` to allow users to not choose a price side.
                # In the if statement below, we translate this choice to one that the `IPA_Equity_Vola_n_Greeeks` funciton will understand.
                if option_price_side_choice.value == "Let Program Choose":
                    option_price_side_choice_val = None
                else:
                    option_price_side_choice_val = option_price_side_choice.value

                ipa_data = IPA_Equity_Vola_n_Greeeks(
                    debug=debug,
                    underlying=eqty_choice.value,
                    strike=None,
                    maturity=calendar.value[0], # "2024-03-15", # calendar.value,
                    maturity_format = '%Y-%m-%d', # e.g.: '%Y-%m-%d', '%Y-%m-%d %H:%M:%S' or '%Y-%m-%dT%H:%M:%SZ'
                    option_type = c_p_choice.value,
                    buy_sell = b_s_choice.value,
                    curr = report_ccy_choice.value,
                    exercise_style = 'EURO',
                    option_price_side = option_price_side_choice_val,
                    underlying_time_stamp = 'Close',
                    resample = '10min',  # You can consider this the 'bucket' or 'candles' from which calculations will be made.
                    rsk_free_rate_prct = rsk_free_rate_prct_choice.value, # for `".SPX"`, I go with `'USDCFCFCTSA3M='`; for `".STOXX50E"`, I go with `'EURIBOR3MD='`
                    rsk_free_rate_prct_field = 'TR.FIXINGVALUE' # for `".SPX"`, I go with `'TR.FIXINGVALUE'`; for `".STOXX50E"`, I go with `'TR.FIXINGVALUE'` too.
                    ).initiate().get_data()

                sngl_fig, worked = ipa_data.graph(
                    title=ipa_data.ipa_df_gmt_no_na.columns.name).fig, True

                strikes_lst, undrlying_optn_ric_lst, df_gmt_lst, df_lst, fig_lst = ipa_data.cross_moneyness(
                    smile_range=0) # We alter our Smile function for our use without smiles.

                if debug:
                    print(strikes_lst)
                    display(fig_lst[0])
                    display(fig_lst[-1])

                volatility_result = pd.concat([i.Volatility for i in df_lst], axis=1, join="outer")
                volatility_result.columns = [str(int(i)) for i in strikes_lst]
                volatility_result.index.name = "ImpliedVolatilities"
                volatility_result

                df = volatility_result.copy()
                # Assuming df is your DataFrame and 'timestamp' is your time column
                df['timestamp'] = df.index
                df.timestamp = pd.to_datetime(df.timestamp)
                df.set_index('timestamp', inplace=True)

                # Resample to daily data and compute daily averages
                daily_df = df.resample('7D').mean()

                # Fill NA/NaN values using the specified method
                daily_df_filled = daily_df.fillna(np.nan).astype(float).dropna()
                daily_df_filled.index = [str(i) for i in daily_df_filled.index]
                daily_df_filled = daily_df_filled.T

                # Let's go back to the `sngl_fig` figure created above
                if worked:
                    IPython.display.clear_output(wait=True)
                sngl_fig.show()

    # refister click handler for button
    print("\n")
    button.on_click(click_handler)
    display(button)

    # display our widgets
    display(button_output)


In [ ]:
Eqty_ATM_Optn_Vola_n_Greeks(debug=False)